<a href="https://colab.research.google.com/github/ziatdinovmax/gpax/blob/main/examples/simpleGP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compare SimpleGP and viGP

This is a simple notebook to compare timings and results of two different commonly used GPs. One trained using NUTS, and the other trained using SVI.

*Prepared by Matthew R. Carbone & Maxim Ziatdinov (2023)*

## Install & Import

Install GPax package:

In [ ]:
!pip install -q git+https://github.com/ziatdinovmax/gpax.git

Import needed packages:

In [ ]:
try:
    # For use on Google Colab
    import gpax

except ImportError:
    # For use locally (where you're using the local version of gpax)
    print("Assuming notebook is being run locally, importing local gpax module")
    import sys
    sys.path.append("..")
    import gpax

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

gpax.utils.enable_x64()  # enable double precision

## Create data

Generate some noisy observations:

In [ ]:
np.random.seed(0)

NUM_INIT_POINTS = 25 # number of observation points
NOISE_LEVEL = 0.1 # noise level

# Generate noisy data from a known function
f = lambda x: np.sin(10*x)

X = np.random.uniform(-1., 1., NUM_INIT_POINTS)
y = f(X) + np.random.normal(0., NOISE_LEVEL, NUM_INIT_POINTS)

# Plot generated data
plt.figure(dpi=100)
plt.xlabel("$x$")
plt.ylabel("$y$")
plt.scatter(X, y, marker='x', c='k', zorder=1, label='Noisy observations')
plt.ylim(-1.8, 2.2);

## Standard `ExactGP`

Next, we initialize and train a GP model. We are going to use an RBF kernel, $k_{RBF}=𝜎exp(-\frac{||x_i-x_j||^2}{2l^2})$, which is a "go-to" kernel functions in GP.

In [ ]:
# Get random number generator keys for training and prediction
rng_key, rng_key_predict = gpax.utils.get_keys()

# Initialize model
gp_model_1 = gpax.ExactGP(1, kernel='RBF')

# Run Hamiltonian Monte Carlo to obtain posterior samples for kernel parameters and model noise
gp_model_1.fit(rng_key, X, y, num_chains=1)

## Standard `viGP`

In [ ]:
# Get random number generator keys for training and prediction
rng_key, rng_key_predict = gpax.utils.get_keys()

# Initialize model
gp_model_2 = gpax.viGP(1, kernel='RBF')

# Run Hamiltonian Monte Carlo to obtain posterior samples for kernel parameters and model noise
gp_model_2.fit(rng_key, X, y)

In [ ]:
X_test = np.linspace(-1, 1, 100)

In [ ]:
y_pred_1, y_sampled_1 = gp_model_1.predict(rng_key_predict, X_test, n=200)

In [ ]:
y_pred_2, y_sampled_2 = gp_model_2.predict(rng_key_predict, X_test, n=200)

Note that SVI (the `viGP`) is significantly faster. SVI is usually better to use on larger datasets and is more easily scalable. In this case, they produce similar results.

In [ ]:
y_sampled_1.shape

In [ ]:
y_sampled_2.shape  # Note shape difference between predict methods

Plot the obtained results:

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(6, 2), dpi=200)

ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.plot(X_test, y_pred_1, lw=1.5, zorder=2, c='r', label='NUTS/MCMC')
ax.fill_between(X_test, y_pred_1 - y_sampled_1.std(axis=(0,1)), y_pred_1 + y_sampled_1.std(axis=(0,1)),
                color='r', alpha=0.3, linewidth=0)


ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.plot(X_test, y_pred_2, lw=1.5, zorder=2, c='b', label='SVI')
ax.fill_between(X_test, y_pred_2 - np.sqrt(y_sampled_2), y_pred_2 + np.sqrt(y_sampled_2),
                color='b', alpha=0.3, linewidth=0)



ax.set_ylim(-1.8, 2.2)

ax.scatter(X, y, marker='x', c='k', zorder=2, label="Noisy observations", alpha=0.7)

ax.legend(loc='upper left', ncols=3)

plt.show()